In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds


In [ ]:
movies_df=pd.read_csv('movies.csv')
ratings_df=pd.read_csv('ratings.csv')

movies_df.head(10)

In [ ]:
Rating_df=ratings_df.pivot(index='userId',columns='movieId',values='rating').fillna(0)

Rating_df.shape

In [ ]:
Rate=Rating_df.values
ratings_mean=np.mean(Rate,axis=1)
R_demean = Rate - ratings_mean.reshape(-1,1)

In [ ]:
U_matrix, sigma_values, Vtrans_matrix = svds(R_demean, k = 50)
sigma_values

In [ ]:
sigma_matrix = np.diag(sigma_values)

sigma_matrix.shape

In [ ]:
user_pred_ratings = np.dot(np.dot(U_matrix, sigma_matrix), Vtrans_matrix) + ratings_mean.reshape(-1, 1)
predicted_df = pd.DataFrame(user_pred_ratings, columns = Rating_df.columns)

In [ ]:
def recommend_movies(predictions_df, userId, movies_df, ratings_df, no_of_recommendations=5):
    user_number = userId - 1 
    predictions_values = predictions_df.iloc[user_number]

    user_ratings =ratings_df[ratings_df.userId == (userId)]
    user_rated_movies = (user_ratings.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User',userId,' has already rated',user_rated_movies.shape[0],' movies.' )
    print ('Recommending the highest',no_of_recommendations,' predicted ratings movies not already rated.')
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommend = (movies_df[~movies_df['movieId'].isin(user_rated_movies['movieId'])].
         merge(pd.DataFrame(predictions_values).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').                   
         rename(columns = {user_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:no_of_recommendations, :-1]
                      )
    return  user_rated_movies, recommend

previously_rated, recommendations = recommend_movies(predicted_df,500, movies_df, ratings_df, 10)

In [ ]:
previously_rated

In [ ]:
recommendations